In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

from geopy import distance


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline

In [2]:
path = './RCdata'

perfix = '' 
file_name =  '' 

os.chdir(path)

## Load datasets

In [3]:

df_rating_csv = pd.read_csv('Rating_Final_RECODED.csv', sep=',', header='infer') #, index_col= 0
df_rating_csv

,Unnamed: 0,userID,placeID,rating,food_rating,service_rating,general_score,general_score_binary
0,1,U1077,135085,2,2,2,6,1
1,2,U1077,135038,2,2,1,5,1
2,3,U1077,132825,2,2,2,6,1
3,4,U1077,135060,1,2,2,5,1
4,5,U1068,135104,1,1,2,4,1
...,...,...,...,...,...,...,...,...
1156,1157,U1043,132630,1,1,1,3,1
1157,1158,U1011,132715,1,1,0,2,1
1158,1159,U1068,132733,1,1,0,2,1
1159,1160,U1068,132594,1,1,1,3,1


In [4]:
df_parking = pd.read_csv('chefmozparking.csv', sep=',', header='infer') #, index_col= 0
df_parking

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none
...,...,...
697,132024,street
698,132023,fee
699,132023,public
700,132012,public


In [5]:
col_select = ['placeID', 'latitude', 'longitude', 'price', 'alcohol', 'smoking_area', 'dress_code', 
              'accessibility', 'Rambience', 'franchise', 'other_services']
col_dtypes = {'placeID': int,'latitude': float, 'longitude': float, 'price': str}


df_place = pd.read_csv('geoplaces2.csv', sep=',', usecols = col_select, header='infer', dtype = col_dtypes, encoding = "ISO-8859-1") #, index_col= 0

df_place.rename(columns={"latitude":"place_lat", 'longitude':'place_lon'}, inplace=True)
df_place


,placeID,place_lat,place_lon,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,other_services
0,134999,18.915421,-99.184871,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,none
1,132825,22.147392,-100.983092,No_Alcohol_Served,none,informal,completely,low,familiar,f,none
2,135106,22.149709,-100.976093,Wine-Beer,only at bar,informal,partially,medium,familiar,f,none
3,132667,23.752697,-99.163359,No_Alcohol_Served,none,informal,completely,low,familiar,t,none
4,132613,23.752903,-99.165076,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,none
...,...,...,...,...,...,...,...,...,...,...,...
125,132866,22.141220,-100.931311,No_Alcohol_Served,not permitted,informal,completely,medium,familiar,f,none
126,135072,22.149192,-101.002936,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,none
127,135109,18.921785,-99.235350,Wine-Beer,not permitted,informal,no_accessibility,medium,quiet,f,Internet
128,135019,18.875011,-99.159422,No_Alcohol_Served,none,informal,completely,low,familiar,f,none


In [6]:
col_select = ['userID', 'latitude', 'longitude', 'budget']
col_dtypes = {'placeID': str,'latitude': float, 'longitude': float, 'budget': str}


# smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height

df_user = pd.read_csv('userprofile.csv', sep=',', usecols = col_select, header='infer', dtype = col_dtypes) #, index_col= 0
df_user.rename(columns={"latitude":"user_lat", 'longitude':'user_lon'}, inplace=True)

df_user

,userID,user_lat,user_lon,budget
0,U1001,22.139997,-100.978803,medium
1,U1002,22.150087,-100.983325,low
2,U1003,22.119847,-100.946527,low
3,U1004,18.867000,-99.183000,medium
4,U1005,22.183477,-100.959891,medium
...,...,...,...,...
133,U1134,22.149654,-100.998610,medium
134,U1135,22.170396,-100.949936,low
135,U1136,22.149607,-100.997235,low
136,U1137,22.144803,-100.944623,low


In [7]:
df_place = pd.merge(df_place, df_parking, on= 'placeID', how="left",)

In [8]:
df_place.isnull().sum()

placeID           0
place_lat         0
place_lon         0
alcohol           0
smoking_area      0
dress_code        0
accessibility     0
price             0
Rambience         0
franchise         0
other_services    0
parking_lot       0
dtype: int64

In [9]:
df_rating_user = pd.merge(df_rating_csv, df_user, on= 'userID', how="left",)
df_rating_user.isnull().sum()

Unnamed: 0              0
userID                  0
placeID                 0
rating                  0
food_rating             0
service_rating          0
general_score           0
general_score_binary    0
user_lat                0
user_lon                0
budget                  0
dtype: int64

In [10]:
df_rating_user_place = pd.merge(df_rating_user, df_place, on= 'placeID', how="left",)
df_rating_user_place

,Unnamed: 0,userID,placeID,rating,food_rating,service_rating,general_score,general_score_binary,user_lat,user_lon,...,place_lon,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,other_services,parking_lot
0,1,U1077,135085,2,2,2,6,1,22.156469,-100.985540,...,-100.982680,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,f,none,public
1,2,U1077,135038,2,2,1,5,1,22.156469,-100.985540,...,-100.977767,No_Alcohol_Served,section,informal,no_accessibility,medium,familiar,f,none,none
2,3,U1077,132825,2,2,2,6,1,22.156469,-100.985540,...,-100.983092,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none
3,4,U1077,135060,1,2,2,5,1,22.156469,-100.985540,...,-100.978485,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,none,none
4,5,U1068,135104,1,1,2,4,1,23.752269,-99.168605,...,-99.168434,Full_Bar,not permitted,informal,completely,medium,familiar,t,variety,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,1157,U1043,132630,1,1,1,3,1,23.771030,-99.167082,...,-99.164473,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none
1157,1158,U1011,132715,1,1,0,2,1,23.724972,-99.152856,...,-99.158660,No_Alcohol_Served,none,informal,no_accessibility,low,quiet,f,none,none
1158,1159,U1068,132733,1,1,0,2,1,23.752269,-99.168605,...,-99.162565,No_Alcohol_Served,not permitted,informal,completely,medium,familiar,t,none,yes
1159,1160,U1068,132594,1,1,1,3,1,23.752269,-99.168605,...,-99.165709,No_Alcohol_Served,not permitted,informal,completely,low,familiar,f,none,public


In [11]:
df_rating_user_place.isnull().sum()

Unnamed: 0              0
userID                  0
placeID                 0
rating                  0
food_rating             0
service_rating          0
general_score           0
general_score_binary    0
user_lat                0
user_lon                0
budget                  0
place_lat               0
place_lon               0
alcohol                 0
smoking_area            0
dress_code              0
accessibility           0
price                   0
Rambience               0
franchise               0
other_services          0
parking_lot             0
dtype: int64

In [12]:
df_rating_user_place['distance_km'] = df_rating_user_place.apply(
    lambda row: distance.geodesic([row.user_lat, row.user_lon], [row.place_lat, row.place_lon]).km, axis=1)

In [13]:
df_rating_user_place

,Unnamed: 0,userID,placeID,rating,food_rating,service_rating,general_score,general_score_binary,user_lat,user_lon,...,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,other_services,parking_lot,distance_km
0,1,U1077,135085,2,2,2,6,1,22.156469,-100.985540,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,f,none,public,0.693407
1,2,U1077,135038,2,2,1,5,1,22.156469,-100.985540,...,No_Alcohol_Served,section,informal,no_accessibility,medium,familiar,f,none,none,0.806876
2,3,U1077,132825,2,2,2,6,1,22.156469,-100.985540,...,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none,1.036330
3,4,U1077,135060,1,2,2,5,1,22.156469,-100.985540,...,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,none,none,0.729154
4,5,U1068,135104,1,1,2,4,1,23.752269,-99.168605,...,Full_Bar,not permitted,informal,completely,medium,familiar,t,variety,yes,0.080878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,1157,U1043,132630,1,1,1,3,1,23.771030,-99.167082,...,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none,2.022179
1157,1158,U1011,132715,1,1,0,2,1,23.724972,-99.152856,...,No_Alcohol_Served,none,informal,no_accessibility,low,quiet,f,none,none,1.015471
1158,1159,U1068,132733,1,1,0,2,1,23.752269,-99.168605,...,No_Alcohol_Served,not permitted,informal,completely,medium,familiar,t,none,yes,0.617608
1159,1160,U1068,132594,1,1,1,3,1,23.752269,-99.168605,...,No_Alcohol_Served,not permitted,informal,completely,low,familiar,f,none,public,0.295448


In [14]:
df = df_rating_user_place.drop(['Unnamed: 0', 'userID', 'placeID', 'rating', 'food_rating', 'service_rating', 'general_score', 
                                'user_lat', 'user_lon', 'place_lat', 'place_lon'],
                              axis=1)

In [15]:
df

,general_score_binary,budget,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,other_services,parking_lot,distance_km
0,1,medium,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,f,none,public,0.693407
1,1,medium,No_Alcohol_Served,section,informal,no_accessibility,medium,familiar,f,none,none,0.806876
2,1,medium,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none,1.036330
3,1,medium,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,none,none,0.729154
4,1,low,Full_Bar,not permitted,informal,completely,medium,familiar,t,variety,yes,0.080878
...,...,...,...,...,...,...,...,...,...,...,...,...
1156,1,medium,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none,2.022179
1157,1,medium,No_Alcohol_Served,none,informal,no_accessibility,low,quiet,f,none,none,1.015471
1158,1,low,No_Alcohol_Served,not permitted,informal,completely,medium,familiar,t,none,yes,0.617608
1159,1,low,No_Alcohol_Served,not permitted,informal,completely,low,familiar,f,none,public,0.295448


In [16]:
df.to_csv('selected_data.csv', index=False)

In [17]:
df = pd.read_csv('selected_data.csv')
df

,general_score_binary,budget,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,other_services,parking_lot,distance_km
0,1,medium,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar,f,none,public,0.693407
1,1,medium,No_Alcohol_Served,section,informal,no_accessibility,medium,familiar,f,none,none,0.806876
2,1,medium,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none,1.036330
3,1,medium,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,none,none,0.729154
4,1,low,Full_Bar,not permitted,informal,completely,medium,familiar,t,variety,yes,0.080878
...,...,...,...,...,...,...,...,...,...,...,...,...
1156,1,medium,No_Alcohol_Served,none,informal,completely,low,familiar,f,none,none,2.022179
1157,1,medium,No_Alcohol_Served,none,informal,no_accessibility,low,quiet,f,none,none,1.015471
1158,1,low,No_Alcohol_Served,not permitted,informal,completely,medium,familiar,t,none,yes,0.617608
1159,1,low,No_Alcohol_Served,not permitted,informal,completely,low,familiar,f,none,public,0.295448
